### Acceso a drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Bibliotecas 

In [ ]:
!sudo apt install tesseract-ocr
!pip3 install pytesseract
!cp drive/MyDrive/HackathonRIIAA2021/Data/spa.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

!apt install enchant
!pip install pyenchant
!sudo apt-get install myspell-es

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
enchant is already the newest version (1.6.0-11.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
myspell-es is already the newest version (1.11-14).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remov

In [ ]:
import os 
import re
import string
import pytesseract as ocr
import cv2 as cv
import numpy as np
from tqdm import tqdm
from enchant.checker import SpellChecker
from difflib import SequenceMatcher

In [ ]:
# cleanup text
def get_personslist(text):
    personslist=[]
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'PERSON':
                personslist.insert(0, (chunk.leaves()[0][0]))
    return list(set(personslist))


# using enchant.checker.SpellChecker, identify incorrect words
def identify_incorrect_words(text):
  rep = { '\n': ' ', '\\': ' ', '\"': '"', '-': ' ', '"': ' " ', 
        '"': ' " ', '"': ' " ', ',':' , ', '.':' . ', '!':' ! ', 
        '?':' ? ' , '*':' * ', 
        '(': ' ( ', ')': ' ) ', '=-\n':''}
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
  # personslist = get_personslist(text)
  # print("PERSON LIST")
  # ignorewords = personslist + ["!", ",", ".", "\"", "?", '(', ')', '*', '`']
  ignorewords = ["!", ",", ".", "\"", "?", '(', ')', '*']
  spell = SpellChecker("es_MX")
  words = text.split()
  incorrectwords = [w for w in words if not spell.check(w) and w not in ignorewords and len(w) > 1]
  # print("Incorrect WOrds: ", len(incorrectwords))
  # using enchant.checker.SpellChecker, get suggested replacements
  suggestedwords = [spell.suggest(w) for w in incorrectwords]
  # print("Suggested WOrds: ", len(suggestedwords))
  # replace incorrect words with [MASK]
  for w in incorrectwords:
    # print(w)
    # print(text)
    text = text.replace(" " + w + " ", ' [MASK] ')
    # print(text)
  return text, suggestedwords

In [ ]:
!pip install transformers
import torch
from transformers import BertTokenizer,BertForMaskedLM
print('Loading BERT tokenizer...')


Loading BERT tokenizer...


In [ ]:
def get_bert_predictions(text, suggestedwords):
  # Load, train and predict using pre-trained model
  tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

  #Remove punctuations
  punctuations = string.punctuation
  punctuations = punctuations.replace('[', '')
  punctuations = punctuations.replace(']', '')
  for c in text:
    if c in punctuations:
        text = text.replace(c, "")
  print("Text:", text)
  text = '[CLS] ' + text + ' [SEP]'
  tokenized_text = tokenizer.tokenize(text)
  print("Tokenized Text", tokenized_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  MASKIDS = [i for i, e in enumerate(tokenized_text) if e == '[MASK]']
  # Create the segments tensors
  segs = [i for i, e in enumerate(tokenized_text) if e == "."]
  segments_ids=[]
  prev=-1
  for k, s in enumerate(segs):
      segments_ids = segments_ids + [k] * (s-prev)
      prev=s
  segments_ids = segments_ids + [len(segs)] * (len(tokenized_text) - len(segments_ids))
  segments_tensors = torch.tensor([segments_ids])
  # prepare Torch inputs 
  tokens_tensor = torch.tensor([indexed_tokens])
  # Load pre-trained model
  model = BertForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
  model.eval()
  
  # Predict all tokens
  with torch.no_grad():
      predictions = model(tokens_tensor, segments_tensors)
  pred_words=[]
  for i in range(len(MASKIDS)):
      # print("Obtener Predicciones i= {}, len = {}, MASKIDS[i]={}, len predictions= {}".format(i, len(MASKIDS), MASKIDS[i], len(predictions[0][0])))
      preds = torch.topk(predictions[0][0][MASKIDS[i]], k=5) 
      # print("tengo las predicciones")
      indices = preds.indices.tolist()
      # print("tengo los indices")
      # print(indices)
      list1 = tokenizer.convert_ids_to_tokens(indices)
      # print("tengo los tokens")
      # print(len(suggestedwords))

      list2 = suggestedwords[i]
      print("Sugerencias SpellChecker", list2)
      print("Sugerencias BERT", list1)
      simmax=0
      predicted_token=''
      for word1 in list1:
          for word2 in list2:
              # print("Word 1:{}, Word 2:{}".format(word1, word2))
              s = SequenceMatcher(None, word1, word2).ratio()
              if s is not None and s > simmax:
                  simmax = s
                  predicted_token = word1
      # print("predicted token:", predicted_token)
      text = text.replace('[MASK]', predicted_token, 1)
      # print("REPLACEEEEEEEEEEEEEEEE=================================================")
      # print(text)
  return text

### Función para procesar imagenes y utilizar Tessearct-OCR

In [ ]:
def get_processed_images(src_path, processed_path, texts_path, bin_option=True):
  source = [f for f in os.listdir(src_path) if os.path.isfile(os.path.join(src_path, f))]
  print('\n[INFO] Se encontraron {} imagenes para procesar.\n'.format(len(source)))
  # for index in tqdm(range(len(source))):
  for index in tqdm(range(1)):
    # Lectura y escalado de imagenes #
    image_name = source[index].split('.')[0]
    image = cv.imread('{}{}'.format(src_path,source[index]))
    image = cv.resize(image, None, fx=1.3, fy=1.3, interpolation=cv.INTER_CUBIC)
    """
    Preprocesamiento de imagenes
    """
    # Eliminación de ruido causado por sombras en las imagenes #
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    dilated_image = cv.dilate(gray_image, np.ones((7,7), np.uint8))
    blur_image = cv.medianBlur(dilated_image,21)
    diff_image = 255 - cv.absdiff(gray_image, blur_image)
    norm_image = diff_image.copy()
    cv.normalize(diff_image, norm_image, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8UC1)
    thr_img = cv.threshold(norm_image, 230, 0, cv.THRESH_TRUNC)[1]
    cv.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8UC1)
    bin_image = cv.threshold(thr_img, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
    
    # detección de texto #
    bin_image_inv = cv.threshold(thr_img, 0, 255, cv.THRESH_OTSU | cv.THRESH_BINARY_INV)[1]
    rect_kernel = cv.getStructuringElement(cv.MORPH_RECT, (90, 90))
    dilation = cv.dilate(bin_image_inv, rect_kernel, iterations = 1)
    
    contours, hierarchy = cv.findContours(dilation, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    if bin_option:
        image_copy = bin_image.copy()
    else:
        image_copy = gray_image.copy()
    count = -1
    for cnt in contours:
        count +=1
        x,y,w,h = cv.boundingRect(cnt)
        cropped = image_copy[y:y + h, x:x + w]
        h,w = cropped.shape
        if h>490 and w>490:
            try:
              text = ocr.image_to_string(cropped, lang='spa')
              print("======================Texto OCR:==========================")
              print(text)
              print('\n')
              # print("======================Texto Masked:========================")
              text, suggestedwords = identify_incorrect_words(text)
              # print(text)
              text = get_bert_predictions(text, suggestedwords)
              print("======================Texto BERT======================")
              print(text)
              if len(text) > 0:
                  cv.imwrite('{}{}_processed_{}.png'.format(processed_path,image_name,count), cropped)
                  textfile = open('{}{}_text.txt'.format(texts_path,image_name),'w')
                  textfile.write(text)
                  textfile.close()
            except:
              print(e)
              continue

In [ ]:
IMAGES_PATH = 'drive/MyDrive/Datos - Hackathon JusticIA/Fichas_auto/'
PROCESSED_PATH = 'drive/MyDrive/HackathonRIIAA2021/Processed_images/Fichas_auto/'
TEXTS_PATH = 'drive/MyDrive/HackathonRIIAA2021/Texts/Fichas_auto/'

In [ ]:
get_processed_images(src_path=IMAGES_PATH, processed_path=PROCESSED_PATH, texts_path=TEXTS_PATH)


[INFO] Se encontraron 1000 imagenes para procesar.



  0%|          | 0/1 [00:00<?, ?it/s]

======================Texto OCR:==========================
dgric.y tomendo en consideración el fallo del Jue,
que nulificó las elecciones en cuestión,declare -
acéfala la actual directiva de la Unión y convoqui
a nuevos Comicios..eEn la C.NoC.,Seé hacen comenta=-
rios en el sentido de que el descontento correro-
es utilizado-en contra del diricente mÁáximo de -
esa Central<,para que- éste no llegue al Cong.Ca -
1a Unión9y que el Srio.Cral.de la Loco.éode GrOo. o
ha manifestado que él,an lo personal,no está teme
roso del problema coprero,porque c-en a con le ayl
da del Dip.Loc.LIC RUBEN ZUNO ARCE,gXpÍ)_%Q¿%35D96'Z
- . 2 lg o
Ag to9-967.> Se tiene conocimie nto de que cl d'ia a
h de los corrientes, el Dio, Fod,., CuSaR DEL Alloo
GEL FUENTES, tieno ponsado salir con destino a .o
Acapulcos Groo; con cl objeto de scoguir ilcvando

TA EE NAA EA D s - MA c —
NA Ae — en o a.
— a



Text: dgric  y [MASK] en consideración el fallo del [MASK]   que [MASK] las elecciones en cuestión  declare   ac

100%|██████████| 1/1 [00:15<00:00, 15.23s/it]

======================Texto BERT======================
[CLS] dgric  y basado en consideración el fallo del gobierno   que de las elecciones en cuestión  declare   acéfala la actual directiva de la Unión y no a nuevos Comicios    sobre la C  de    c hacen comentarios en el sentido de que el descontento se  es utilizado en contra del gobierno presidente de   esa empresa  para que  éste no llegue al el  Ca   a y que el gobierno  gobierno  de la Loco  el de   o ha manifestado que él  a lo personal  no está teme roso del problema de  porque c en a con le le da del el    e  [UNK] ARCE  [UNK]  es      2  o   967   Se tiene el conocimiento de que a el a h de los corrientes   el Dio   a       d DEL gobierno GEL FUENTES    puede salir con destino a   o a con con con objeto de hacer el  TA la [UNK] EA D s   MA c — NA [UNK] — en o a   — a  [SEP]
